In [73]:
import sys
sys.path.append('/media/data/hungnt/work/SourceCode/Brain_DNA/src/models/')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle

from Model import Impf_DNAMLP

In [74]:
fold = '1.0'
test_fold = '1.0'
class_name = 'Embryonal'

In [75]:
cfg = config.config_dict
impf_cfg = config.important_features_config_dict
TRAIN_CSV_DIR = cfg['BIN_TRAIN_CSV_DIR']
TEST_CSV_DIR = cfg['BIN_TEST_CSV_DIR']
FOLD_CSV_FILE = os.path.join(TRAIN_CSV_DIR, class_name, f'{fold}_train.csv')
TEST_FOLD_CSV_FILE = os.path.join(TEST_CSV_DIR, class_name, f'{test_fold}_test.csv')
device = impf_cfg['device']
device

'cuda'

In [76]:
impf_selector = 'RF'

if impf_selector.lower() == 'rf':
    selector_name = 'RF'
    print('Impf selector: RF')
elif impf_selector.lower() == 'mlp':
    selector_name = 'MLP'
    print('Impf selector: MLP')
elif impf_selector.lower() == 'lr':
    print('Impf selector: LR')
    selector_name = 'LR'
else:
    print('Default impf selector: RF')
    selector_name = 'RF'
    
with open(os.path.join(impf_cfg[f'{selector_name}_IMPF_DIR'], class_name, f'{fold}_impf.pkl'), 'rb') as handle:
    impf = pickle.load(handle)

Impf selector: RF


In [77]:
df_test = pd.read_csv(TEST_FOLD_CSV_FILE, index_col=0)
df_test.shape

(597, 10002)

In [78]:
rand = np.random.randint(df_test.shape[0])
sample = df_test.iloc[rand,:].loc[impf]
df_test.iloc[rand, -2:], sample

(label             MB, G3
 class_group    Embryonal
 Name: 7970376029_R03C01, dtype: object,
 cg26772540    0.773156
 cg07254421     0.84622
 cg19327049    0.155361
 cg26824216    0.667186
 cg20397902    0.175935
 cg24441810    0.111134
 cg25459300    0.290347
 cg03556243    0.817975
 cg17781789    0.096427
 cg17777592    0.867769
 Name: 7970376029_R03C01, dtype: object)

In [79]:
sample_features = np.expand_dims(np.array(sample).astype(float), axis = 0)
sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

(array([[0.77315596, 0.84622041, 0.15536105, 0.66718553, 0.1759347 ,
         0.11113393, 0.29034679, 0.81797456, 0.09642658, 0.86776894]]),
 numpy.ndarray,
 (1, 10),
 numpy.float64)

In [80]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['IMPF_MLP_BEST_STATES_DIR'], class_name, f'{fold}_best_state.pth')

in_features = impf_cfg['N_IMPF']
n_classes = impf_cfg['mlp_n_classes']
in_features = 10

model = Impf_DNAMLP(in_features, n_classes)
model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [81]:
sample_features.shape

(1, 10)

In [82]:
logits = model((torch.Tensor(sample_features).to(device)))
logits

tensor([[0.0882, 0.4649]], device='cuda:0', grad_fn=<AddmmBackward0>)